# Progressive Growing of GANs For Improved Quality, Stability and Variation

## Abstract

무엇을 해냈나?

new training methodology for GAN. ***G와 D를 서서히 키워가는 것***

→ training 속도 ↑ & 안정화

생성 이미지의 variation을 증가시키는 simple way

GAN의 결과를 evalutate할 새로운 측정법

## Introduction

Generative Methods 

ex) Autoregressive models, Variational Autoencoders (VAE), Generative Adversarial Networks (GAN)

- Autoregressive models
    
    ex) PixelCNN
    
    선명한 이미지를 만들지만 검증이 느리고 곧바로 conditional distribution을 만들기 때문에 latent representation이 없음 → 응용가능성 ↓
    
- VAE
    
    train하기 쉽지만 blurry results
    
- GAN
    
    선명한 이미지를 만들지만 small resolution(저해상도), limited variation에서만
    
    더 높은 해상도가 될수록 
    
    생성된 이미지가 D에게 들킬 확률 ↑
    
    메모리의 한계 → smaller minibatches를 사용해야함 → train 불안정함
    

이 중 GAN의 방법을 더 발전시켜 고화질의 이미지를 생성할 수 있다!

## Progressive Growing of GANs

<img src="https://user-images.githubusercontent.com/66329748/169148464-e9beea3b-9469-4435-a3b0-1c2f6aca1c7c.png">

![https://miro.medium.com/max/1400/1*tUhgr3m54Qc80GU2BkaOiQ.gif](https://miro.medium.com/max/1400/1*tUhgr3m54Qc80GU2BkaOiQ.gif)

저해상도의 image에서 시작해, network에 layer를 추가하여 해상도 ↑

ex) 4X4 layer에서 충분히 학습이 잘 되었다고 생각하면 해상도를 올려서 8X8에서 학습

→ large-scale structure (전반적인 구도, 전반적인 형태) 부터 잡아내고 finer scale detail로 shift attention

G와 D가 서로 거울상 & 동시에 grow

새로운 layer가 추가될 때, smooth하게 fade in → 이미 잘 학습된 기존의 layer들의 sudden shock을 방지

![a에서 b로 갈 때 16X16으로 만들어낸 이미지를 단순히 크기만 2배로 키움 (이미지 1)
32 X 32에서 이미지를 만들어냄 (이미지 2)
(1-a) * (이미지 1) + a * (이미지 2) = 생성 이미지
a는 0에서 1로 점차 증가. 즉, 이전 layer의 영향력은 점점 작아지고 새로운 layer의 영향은 점점 커짐](https://user-images.githubusercontent.com/66329748/169148651-6c3d5622-1cae-413c-8846-99c0378139e6.png)

a에서 b로 갈 때 16X16으로 만들어낸 이미지를 단순히 크기만 2배로 키움 (이미지 1)
32 X 32에서 이미지를 만들어냄 (이미지 2)
(1-a) * (이미지 1) + a * (이미지 2) = 생성 이미지
a는 0에서 1로 점차 증가. 즉, 이전 layer의 영향력은 점점 작아지고 새로운 layer의 영향은 점점 커짐

progressive training의 몇가지 장점

- 작은 이미지 생성은 stable → 거기부터 해상도를 점점 키워가는 것이 latent vector로부터 만들어내는 것보다 much simpler question
    
    WGAN-GP Loss 나 LSGAN Loss를 사용
    
- training time ↓
    
    대부분의 iteration이 낮은 해상도일 때 일어남
    
    약 2~6배 빠름
    

## Increasing Variation Using Minibatch Standard Deviation

GAN은 training data의 부분적인 variation만 capture하는 경향이 있다 → 해결책 : “minibatch discrimination” (Salimans et al. “Improved techniques for training GANs”. 2016)

individual images가 아니라 minibatch 단위에서 feature statistics를 계산 → 생성이미지와 train 이미지가 similar statistics 갖게함

minibatch layer를 Discriminator 끝에 넣음

이 논문에서 제시하는 더 simple한 solution

minibatch의 각 spatial location의 각 feature들의 표준편차를 계산 (N X C X H X W → C X H X W)

spatial location들과 모든 feature에 대한 이 추정치들의 평균을 계산하여 single value로 만듦 (C X H X W → 1 X H X W)

값을 복제하고 모든 spatial location과 모든 feature들에 연결하여 하나의 추가적인 feature map을 생성

이 layer는 D의 어디든 들어갈 수 있지만 끝부분에 넣는게 best

## Normalization in Generator and Discriminator

GAN은 G와 D의 분필요한 경쟁으로 인해 발생하는 escalation of signal magnitude에 취약 

→ 보통은 이런 문제점을 해결하기 위해 Batch Normalization 사용 → Covariance Shift 방지

→ 하지만 이 논문에서는 Covariance Shift를 방지할 필요 X 

> *we have not observed that to be an issue in GANs*
> 

→ Different Approach

### Equalized Learning Rate

기존의 신중한 weight initialization 대신, 단순히 N(0, 1) initialization 사용 & runtime 중에 weight scale

![스크린샷 2022-05-19 오전 3.10.56.png](https://user-images.githubusercontent.com/66329748/169148759-582ee2ac-a548-4381-8d89-eb42eea58fbc.png)

$w_i$는 weight, $c$는 Kaiming He Initialization의 per-layer normalization constant

기존의 RMSProp이나 Adam과 같은 adaptive SGD 방식은 어떤 parameter들이 더 넓은 dynamic range를 가지면 적응하는데 시간이 오래 걸림

이 논문에서는 dynamic range를 모든 weight에 대해 같게 함으로써 learning speed도 동일하게 함

### Pixelwise Feature Vector Normalization in Generator

위에서 언급한 Different Approach → “local response normalization” 사용

G의 각 convolutional layer가 끝날 때, 각 pixel의 feature vector를 unit length별로 normalize

→ G를 harm하지 X, 결과를 변형시키지 X, 단지 escalation of signal magnitude 방지

## Mulit-Scale Statistical Similarity for Assessing GAN Results

GAN의 결과를 다른 것과 비교하려면 많은 수의 이미지들을 분석해야함 → 힘듦

MS-SSIM이라는 방법이 있지만 단점들이 많이 존재 (작은 변화에 반응 X, train data와의 유사도 관점에서 이미지의 quality를 직접적으로 평가 X)

이 논문에서는 Laplacian Pyramid로부터 나오는 local image patch들의 분포간 multiscale statistical similarity를 통해 분석

1. Training set과 Generated set에서 local image patch들을 얻음
2. 둘 사이의 SWD(Sliced Wasserstein Distance)를 계산하여 statistical similarity를 estimate

작은 W distance는 local image patch들의 분포가 비슷하다는 것을 의미

# 모델 구조

![Untitled](https://user-images.githubusercontent.com/66329748/169148853-414b968b-1255-4e5d-a3b9-a67b51603bfe.png)

![Untitled](https://user-images.githubusercontent.com/66329748/169148859-1170b239-970e-4161-ae03-030bcbad22db.png)

In [ ]:
!git clone https://github.com/facebookresearch/pytorch_GAN_zoo.git

Cloning into 'pytorch_GAN_zoo'...
remote: Enumerating objects: 1523, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1523 (delta 17), reused 21 (delta 13), pack-reused 1495
Receiving objects: 100% (1523/1523), 2.06 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (977/977), done.


In [ ]:
    # Generator 부분
    
    def addScale(self, depthNewScale):
        r"""
        Add a new scale to the model. Increasing the output resolution by
        a factor 2
        Args:
            - depthNewScale (int): depth of each conv layer of the new scale
        """
        depthLastScale = self.scalesDepth[-1]

        self.scalesDepth.append(depthNewScale)

        self.scaleLayers.append(nn.ModuleList())

        self.scaleLayers[-1].append(EqualizedConv2d(depthLastScale,
                                                    depthNewScale,
                                                    3,
                                                    padding=1,
                                                    equalized=self.equalizedlR,
                                                    initBiasToZero=self.initBiasToZero))
        self.scaleLayers[-1].append(EqualizedConv2d(depthNewScale, depthNewScale,
                                                    3, padding=1,
                                                    equalized=self.equalizedlR,
                                                    initBiasToZero=self.initBiasToZero))

        self.toRGBLayers.append(EqualizedConv2d(depthNewScale,
                                                self.dimOutput,
                                                1,
                                                equalized=self.equalizedlR,
                                                initBiasToZero=self.initBiasToZero))

    def setNewAlpha(self, alpha):
        r"""
        Update the value of the merging factor alpha
        Args:
            - alpha (float): merging factor, must be in [0, 1]
        """

        if alpha < 0 or alpha > 1:
            raise ValueError("alpha must be in [0,1]")

        if not self.toRGBLayers:
            raise AttributeError("Can't set an alpha layer if only the scale 0"
                                 "is defined")

        self.alpha = alpha

In [ ]:
def updateAlphaJumps(self, nJumpScale, sizeJumpScale):
        """
        Given the number of iterations between two updates of alpha at each
        scale and the number of updates per scale, build the effective values of
        self.maxIterAtScale and self.alphaJumpVals.
        Args:
            - nJumpScale (list of int): for each scale, the number of times
                                        alpha should be updated
            - sizeJumpScale (list of int): for each scale, the number of
                                           iterations between two updates
        """

        n_scales = min(len(nJumpScale), len(sizeJumpScale))

        for scale in range(n_scales):

            self.modelConfig.iterAlphaJump.append([])
            self.modelConfig.alphaJumpVals.append([])

            if nJumpScale[scale] == 0:
                self.modelConfig.iterAlphaJump[-1].append(0)
                self.modelConfig.alphaJumpVals[-1].append(0.0)
                continue

            diffJump = 1.0 / float(nJumpScale[scale])
            currVal = 1.0
            currIter = 0

            while currVal > 0:

                self.modelConfig.iterAlphaJump[-1].append(currIter)
                self.modelConfig.alphaJumpVals[-1].append(currVal)

                currIter += sizeJumpScale[scale]
                currVal -= diffJump

            self.modelConfig.iterAlphaJump[-1].append(currIter)
            self.modelConfig.alphaJumpVals[-1].append(0.0)

In [ ]:
    def addScale(self, depthNewScale):
        r"""
        Add a new scale to the model. The output resolution becomes twice
        bigger.
        """
        self.netG = self.getOriginalG()
        self.netD = self.getOriginalD()

        self.netG.addScale(depthNewScale)
        self.netD.addScale(depthNewScale)

        self.config.depthOtherScales.append(depthNewScale)

        self.updateSolversDevice()

    def updateAlpha(self, newAlpha):
        r"""
        Update the blending factor alpha.
        Args:
            - alpha (float): blending factor (in [0,1]). 0 means only the
                             highest resolution in considered (no blend), 1
                             means the highest resolution is fully discarded.
        """
        print("Changing alpha to %.3f" % newAlpha)

        self.getOriginalG().setNewAlpha(newAlpha)
        self.getOriginalD().setNewAlpha(newAlpha)

        if self.avgG:
            self.avgG.module.setNewAlpha(newAlpha)

        self.config.alpha = newAlpha